In [1]:
import os
import sys

# Get the absolute path of the notebook's directory
notebook_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(notebook_dir)

In [2]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from src.app import PermutationSolver


In [3]:
n = 16
config = {
            'n_permutations_length': n,
            
            # Random walks params
            'random_walks_type': 'non-backtracking-beam',
            'n_random_walk_length': int(n * (n-1) / 2),
            'n_random_walks_to_generate': 1000,
            'n_random_walks_steps_back_to_ban': 8,
            
            # Neural Net params
            'model_type': 'MLP',
            'list_layers_sizes': [2**12],
            'n_epochs': 100,
            'batch_size': 1024,
            'lr_supervised': 0.001,
            
            # DQN training
            'n_epochs_dqn': 16000,
            'flag_dqn_round': False,
            'n_random_walks_to_generate_dqn': 1000,
            'verbose_loc': 50,
            'lr_rl': 0.001,
            
            # Beam search
            'beam_search_torch': True,
            'beam_search_Fironov': False,
            'beam_width': 1,
            'n_steps_limit': 4 * n**2,
            'alpha_previous_cost_accumulation': 0,
            'beam_search_models_or_heuristics': 'model_torch',
            'ban_p0_p1_transposition_if_p0_lt_p1_ie_already_sorted': False,
            'n_beam_search_steps_back_to_ban': 32,
            
            # What to solve
            'solve_random_or_longest_state': 'solve_LRX_longest',
            'verbose': 100
        }

In [4]:
n_perm_list = [n]
n_epoch_list = [100]
n_epoch_dqn_list = [16000]
batch_size_list = [2**10]
lr_supervised_list = [0.001]
lr_rl_list = [0.001]
beam_width_list = [2**0]
list_layers_sizes = [[2**12]]

config['mode'] = 'single_hard_hinge'
config['w_anchor'] = 0.0
config['w_hinge'] = 1.0

summary_df = pd.DataFrame()

for n_permutations_length in n_perm_list:
    anchor_mode = 0
    solver = PermutationSolver(config)
    solver.config['n_permutations_length'] = n_permutations_length
    for list_layers_size in list_layers_sizes:
        solver.config['list_layers_sizes'] = list_layers_size
        for lr in lr_supervised_list:
            solver.config['lr_supervised'] = lr
            for lr_rl in lr_rl_list:
                solver.config['lr_rl'] = lr_rl
                for batch_size in batch_size_list:
                    solver.config['batch_size'] = batch_size
                    for n_epoch in n_epoch_list:
                        solver.config['n_epochs'] = n_epoch
                        mlp_losses = solver.train_mlp()
                        # save mlp_model
                        torch.save(solver.mlp_model.state_dict(), f'models/mlp_model_{n_permutations_length}_{n_epoch}_{list_layers_sizes[0]}_{lr}_{batch_size}.pth')
                        for n_epoch_dqn in n_epoch_dqn_list:
                            solver.config['n_epochs_dqn'] = n_epoch_dqn
                            dqn_losses = solver.train_dqn()
                            # save dqn_model
                            torch.save(solver.dqn_model.state_dict(), f'models/dqn_model_{n_permutations_length}_{n_epoch_dqn}_{list_layers_sizes[0]}_{lr}_{batch_size}.pth')
                            for beam_width in beam_width_list:
                                solver.config['beam_width'] = beam_width
                                i_step, flag_found_destination, path = solver.test_beam_search()

                                # calculate min and max of difference between solver.y_anchor and y_valid
                                y_valid = solver.dqn_model(solver.X_anchor)
                                diff = solver.y_anchor.cpu().detach().numpy() - y_valid.cpu().detach().numpy().reshape(1, -1)[0]
                                min_diff = diff.min()
                                max_diff = diff.max()
                                std_diff = diff.std()
                                mean_diff = diff.mean()
                                num_elements_less_than_minus_05 = (diff < -0.5).sum()
                                num_elements_larger_than_05 = (diff > 0.5).sum()
                                wrong_predictions = num_elements_less_than_minus_05 + num_elements_larger_than_05
                                percentage_wrong_predictions = wrong_predictions / len(diff)

                                # save summary_df
                                summary_df_cur = pd.DataFrame({'n_permutations_length': n_permutations_length, 
                                                               'anchor':anchor_mode, 'list_layers_sizes': 
                                                               list_layers_sizes, 'lr_supervised': 
                                                               lr, 'lr_rl': lr_rl, 
                                                               'n_epoch': n_epoch, 
                                                               'n_epoch_dqn': n_epoch_dqn, 
                                                               'beam_width': beam_width, 
                                                               'i_step': i_step, 
                                                               'flag_found_destination': flag_found_destination, 
                                                               'mlp_losses': mlp_losses[-1], 
                                                               'dqn_losses': dqn_losses[-1],
                                                               'min_diff': min_diff,
                                                               'max_diff': max_diff,
                                                               'mean_diff': mean_diff,
                                                               'std_diff': std_diff,
                                                               'num_elements_less_than_minus_1': num_elements_less_than_minus_05,
                                                               'num_elements_larger_than_1': num_elements_larger_than_05,
                                                               'percentage_wrong_predictions': percentage_wrong_predictions
                                                               })
                                summary_df = pd.concat([summary_df, summary_df_cur])
                                summary_df.to_csv('models/summary_df.csv', index=False)


Training MLP:  10%|█         | 10/100 [00:04<00:36,  2.50it/s]

Epoch 10, Loss: 292.1363


Training MLP:  20%|██        | 20/100 [00:07<00:28,  2.83it/s]

Epoch 20, Loss: 264.9992


Training MLP:  30%|███       | 30/100 [00:11<00:26,  2.68it/s]

...

Epoch 14000 | Loss: 0.0364 | Hinge: 0.0348 | Anchor: 0.0000 | Times - RW: 0.16s, Bellman: 0.04s, Train: 0.16s


Training DQN:  88%|████████▊ | 14051/16000 [1:55:20<16:26,  1.98it/s]

Epoch 14050 | Loss: 0.0365 | Hinge: 0.0345 | Anchor: 0.0000 | Times - RW: 0.17s, Bellman: 0.04s, Train: 0.19s


Training DQN:  88%|████████▊ | 14062/16000 [1:55:25<15:54,  2.03it/s]


KeyboardInterrupt: 

In [ ]:
summary_df

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import hsv_to_rgb
import ipywidgets as widgets
from IPython.display import display, clear_output


def visualize_tensor_sequence_notebook(tensor_list):
    """
    Интерактивная визуализация в Jupyter Notebook в виде линии шариков:
    - Использует ipywidgets.IntSlider и перерисовывает график в Output
    - tensor_list: список torch.Tensor размера (N, M)
    """
    M = tensor_list[0].shape[0]
    N = len(tensor_list)
    # позиции на линии: равномерные точки по оси X
    xs = np.arange(M)
    ys = np.zeros(M)

    # слайдер и область вывода
    slider = widgets.IntSlider(value=0, min=0, max=N-1, step=1, description='Frame')
    out = widgets.Output()
    
    def plot_frame(idx):
        vals = tensor_list[idx]
        arr = vals.cpu().numpy().astype(float)
        norm = arr / arr.max()
        hsv = np.stack([np.zeros_like(norm), norm, np.ones_like(norm)], axis=1)
        rgb = hsv_to_rgb(hsv)
        
        with out:
            clear_output(wait=True)
            fig, ax = plt.subplots(figsize=(8,2))
            ax.set_aspect('equal')
            ax.axis('off')
            # рисуем шарики вдоль линии
            scatter = ax.scatter(xs, ys, s=800, color=rgb)
            # цифры внутри шариков
            for x, y, v in zip(xs, ys, vals):
                ax.text(x, y, str(v.item()), ha='center', va='center', fontsize=12, color='black')
            # статус 'start' и 'done'
            status = 'start' if idx == 0 else ('done' if idx == N-1 else '')
            ax.text(M/2 - 0.5, 0.3, status, ha='center', va='bottom', fontsize=16)
            # установка границ
            ax.set_xlim(-1, M)
            ax.set_ylim(-1, 1)
            plt.show()

    # подписка на изменение слайдера
    slider.observe(lambda change: plot_frame(change['new']), names='value')
    display(slider, out)
    # отрисовка начального кадра
    plot_frame(0)


visualize_tensor_sequence_notebook(path)